## ライブラリのインポート

In [7]:
from collections import Counter

import pandas as pd
from sudachipy import dictionary, tokenizer
from tqdm import tqdm

## データの読み込み

In [8]:
# プロジェクトデータの読み込み
project_df = pd.read_csv("../data/raw/project-true.csv")
print(f"プロジェクトデータ: {len(project_df)}件")
print(project_df.head())

# ユーザー職歴データの読み込み
user_df = pd.read_csv(
    "../data/raw/user_work_histories.csv"
)
print(f"\nユーザー職歴データ: {len(user_df)}件")
print(user_df.head())

プロジェクトデータ: 1000件
   id          name                                        description  \
0   1    旅行予約システム設計  当プロジェクトでは運用保守、テスト、デプロイ、ドキュメント作成、自動テスト導入やインフラ構築...   
1   2    予約管理システム設計  現プロジェクトにおいてデータベース設計やデプロイなどに取り組んでいる。\n[実施内容]\nデ...   
2   3  ブロックチェーン開発実装  要件定義、要件分析、パフォーマンス改善、自動テスト導入、クラウド移行やセキュリティ強化などの...   
3   4    動画配信サービス改善  技術選定、UI/UX設計、セキュリティ強化、実装、デプロイやデータベース設計などの開発に取り...   
4   5    物流管理システム改善             データベース設計、運用保守やチームマネジメントなどの開発に取り組んでいます。   

   start_date end_date  
0  2024-04-09      NaN  
1  2024-12-22      NaN  
2  2025-05-20      NaN  
3  2024-11-24      NaN  
4  2024-02-22      NaN  

ユーザー職歴データ: 911件
   id  user_id  history_id project_name  \
0   1        1           1     スマホアプリ開発   
1   2        1           2   医療情報システム開発   
2   3        1           3   データ分析基盤の構築   
3   4        1           4   動画配信サービス構築   
4   5        1           5    教育向けアプリ開発   

                                         description          role  \
0    SNSアプリ開発\n[主な担当]\nドキュメント作成\nテスト実施\nイン

## 名詞抽出関数の定義

In [9]:
def extract_nouns(text: str):
    """
    テキストから名詞を抽出する関数
    """
    if pd.isna(text):
        return []

    # full辞書を使用して辞書オブジェクトを生成
    tokenizer_obj = dictionary.Dictionary(dict="full").create()
    # 形態素解析の粒度設定(C: 最も粗い)
    mode = tokenizer.Tokenizer.SplitMode.C

    # 形態素解析を実行
    morphemes = tokenizer_obj.tokenize(str(text), mode)

    nouns = []
    for m in morphemes:
        # 品詞情報を取得
        pos = m.part_of_speech()[0]
        if pos == "名詞":
            nouns.append(m.surface())
    return nouns


# 動作確認
sample_text = "AWSを使ったWebアプリケーション開発プロジェクト"
print(f"サンプルテキスト: {sample_text}")
print(f"抽出された名詞: {extract_nouns(sample_text)}")

サンプルテキスト: AWSを使ったWebアプリケーション開発プロジェクト
抽出された名詞: ['AWS', 'Web', 'アプリケーション', '開発', 'プロジェクト']


## Word2Vec用のコーパス作成

In [11]:
def create_corpus_from_df(df, columns, ability_columns=None, desc="Processing"):
    """
    DataFrameから指定列のテキストを抽出し、名詞のみのリストのリストを作成
    ability_columnsが指定された場合、その列の値が1であれば列名を単語として追加

    Parameters:
    -----------
    df : pd.DataFrame
        対象のDataFrame
    columns : list
        テキスト抽出対象の列名リスト
    ability_columns : list, optional
        アビリティ（スキル）の列名リスト。値が1の場合に列名を追加する
    desc : str
        進行状況バーの説明文

    Returns:
    --------
    corpus : list of list
        各行の名詞リスト
    """
    corpus = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc=desc):
        # 各行の指定列から名詞を抽出
        row_nouns = []
        for col in columns:
            if col in df.columns and pd.notna(row[col]):
                nouns = extract_nouns(str(row[col]))
                row_nouns.extend(nouns)
        
        # アビリティ列の処理
        if ability_columns:
            for col in ability_columns:
                if col in df.columns and row[col] == 1:
                    row_nouns.append(col)

        if row_nouns:  # 空でない場合のみ追加
            corpus.append(row_nouns)

    return corpus


# プロジェクトデータのコーパスを作成
print("プロジェクトデータの処理中...")
project_columns = ["name", "description"]
project_corpus = create_corpus_from_df(
    project_df, project_columns, desc="プロジェクトデータ"
)

# ユーザー職歴のコーパスを作成
print("\nユーザー職歴データの処理中...")
user_columns = ["project_name", "description", "role"]
# アビリティ列を定義
ability_columns = [
    "HTML", "CSS", "JavaScript", "TypeScript", "Python", "Ruby", "PHP", "Java", 
    "C#", "C++", "Go", "Rust", "Swift", "Kotlin", "React", "Vue", "Angular", 
    "Node.js", "Express", "Django", "Flask", "Rails", "Laravel", "Spring", ".NET", 
    "AWS", "Azure", "GCP", "Docker", "Kubernetes", "MySQL", "PostgreSQL", 
    "MongoDB", "Git", "Linux", "Agile", "Scrum"
]
user_corpus = create_corpus_from_df(user_df, user_columns, ability_columns=ability_columns, desc="ユーザー職歴データ")

# 両方を結合
corpus = project_corpus + user_corpus

print(f"\n✓ プロジェクトデータコーパス: {len(project_corpus)}件")
print(f"✓ ユーザー職歴コーパス: {len(user_corpus)}件")
print(f"✓ 結合コーパス: {len(corpus)}件")

print("\n【プロジェクトデータ】最初の2件のサンプル:")
for i, doc in enumerate(project_corpus[:2]):
    print(f"{i + 1}: {doc[:15]}...")

print("\n【ユーザー職歴データ】最初の2件のサンプル:")
for i, doc in enumerate(user_corpus[:2]):
    print(f"{i + 1}: {doc[:15]}...")


プロジェクトデータの処理中...


プロジェクトデータ: 100%|██████████| 1000/1000 [00:28<00:00, 35.50it/s]



ユーザー職歴データの処理中...


ユーザー職歴データ: 100%|██████████| 911/911 [00:36<00:00, 25.30it/s]


✓ プロジェクトデータコーパス: 1000件
✓ ユーザー職歴コーパス: 911件
✓ 結合コーパス: 1911件

【プロジェクトデータ】最初の2件のサンプル:
1: ['旅行', '予約', 'システム設計', 'プロジェクト', '運用', '保守', 'テスト', 'デプロイ', 'ドキュメント', '作成', '自動', 'テスト', '導入', 'インフラ', '構築']...
2: ['予約', '管理', 'システム設計', 'プロジェクト', 'データベース設計', 'デプロイ', '実施', '内容', 'データベース設計', 'デプロイ', '実装', '自動', 'テスト', '導入', '運用']...

【ユーザー職歴データ】最初の2件のサンプル:
1: ['スマホアプリ', '開発', 'SNS', 'アプリ', '開発', '担当', 'ドキュメント', '作成', 'テスト', '実施', 'インフラ', '構築', '技術', '検証', 'プログラマー']...
2: ['医療情報システム', '開発', 'オンプレミス', '環境', 'AWS', 'クラウド', '移行', '主導', '新技術', '調査', 'PoC', 'アーキテクチャ', '改善', '提案', 'バックエンド']...


## コーパスの統計情報

## コーパスをテキストファイルに保存

In [12]:
# コーパスをテキストファイルに保存
save_path_txt = "../data/processed/corpus-true.txt"
with open(save_path_txt, "w", encoding="utf-8") as f:
    for doc in corpus:
        f.write(" ".join(doc) + "\n")
print(f"コーパスをテキストファイルに保存しました: {save_path_txt}")

コーパスをテキストファイルに保存しました: ../data/processed/corpus-true.txt
